Primeiro conseguir separar a imagem por regiao onde identifica a prova e onde marca as questoes

In [8]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# # Carregue a imagem
# image = cv2.imread('.\Leitor\provas_realizadas\scan0000.jpg')
# image_resize = cv2.resize(image, None, fx=0.5, fy=0.5)
# image_copy = cv2.resize(image, None, fx=0.5, fy=0.5)

# # Converta a imagem para tons de cinza
# gray = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)
# blur = cv2.GaussianBlur(gray,(1,1),1000)
# _, thresh = cv2.threshold(blur, 195, 255, cv2.THRESH_BINARY)

# # Detecte os contornos na imagem
# contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# contours = sorted(contours, key=cv2.contourArea, reverse=True) 

# warped_images = []

# for contorno in contours:
#     # Obtém o retângulo delimitador do contorno
#     (x, y, w, h) = cv2.boundingRect(contorno)
    
#     # Verifica se a área é grande o suficiente (evita contornos pequenos indesejados)
#     if cv2.contourArea(contorno) > 1000:
#         # Obtém os pontos dos cantos do retângulo delimitador
#         pts1 = np.float32([[x, y], [x+w, y], [x+w, y+h], [x, y+h]])
        
#         # Define os pontos para onde a região deve ser transformada
#         pts2 = np.float32([[0, 0], [399, 0], [399, 399], [0, 399]])
        
#         # Calcula a matriz de transformação
#         M = cv2.getPerspectiveTransform(pts1, pts2)
        
#         # Aplica a transformação à região de interesse (ROI)
#         warped = cv2.warpPerspective(image_resize, M, (400, 400))
#         warped_images.append(warped)

# # Exibe as imagens transformadas
# for i, warped in enumerate(warped_images):
#     plt.plot(1, len(warped_images), i+1)
#     plt.imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))
#     plt.axis('off')

# plt.show()

# # Exibe outras imagens (se houver)
# cv2.imshow('image_resize', image_resize)
# cv2.imshow('binary_frame', thresh)
# cv2.imshow('image_copy', image_copy)

# cv2.waitKey(0)
# cv2.destroyAllWindows()


import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

def gabaritoA():
    image = cv2.imread('Leitor\prova A\Cartao_resposta___PROVA_A.jpg')
    image_copy = cv2.resize(image, None, fx=0.5, fy=0.5)

    # Converta a imagem para tons de cinza
    gray = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(1,1),1000)
    _, thresh = cv2.threshold(blur, 195, 255, cv2.THRESH_BINARY)

    # Detecte os contornos na imagem
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True) 

    area_minima_desejada = 10
    area_maxima_desejada = 60000

    # Filtrar os contornos com área maior que o valor desejado
    filtered_countourns = [cont for cont in contours if (cv2.contourArea(cont) > area_minima_desejada) and (cv2.contourArea(cont) < area_maxima_desejada)]

    cv2.drawContours(thresh, filtered_countourns, -1, (0, 0, 255), 1)

    nova_imagem = np.zeros_like(thresh)

    # Desenhe os contornos filtrados na nova imagem
    cv2.drawContours(nova_imagem, filtered_countourns, -1, (255, 255, 255), thickness=cv2.FILLED)

    # Salve a nova imagem
    cv2.imshow('nova_imagem.png', nova_imagem)

    maior_contorno = max(filtered_countourns, key=cv2.contourArea)

    # Encontre o ângulo de rotação com base na orientação do maior contorno
    (x, y), (largura, altura), angulo = cv2.minAreaRect(maior_contorno)


    # Rotação da imagem para endireitar
    altura, largura = nova_imagem.shape[:2]

    filtro_nitidez = np.array([[-1, -1, -1],
                            [-1,  9, -1],
                            [-1, -1, -1]])
    imagem_nitidez = cv2.filter2D(nova_imagem, -1, filtro_nitidez)
    cv2.imwrite(f'nitidez.png', imagem_nitidez)
    contornos, _ = cv2.findContours(imagem_nitidez, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    imagem_cont = []
    for i, contorno in enumerate(contornos):
        # Crie uma máscara preta do mesmo tamanho da imagem original
        mask = np.zeros_like(gray)

        # Desenhe o contorno atual na máscara (branco)
        cv2.drawContours(mask, [contorno], 0, 255, thickness=cv2.FILLED)

        # Aplique a máscara à imagem original para obter a imagem do contorno atual
        contorno_imagem = cv2.bitwise_and(imagem_nitidez, imagem_nitidez, mask=mask)
        # Mostre ou salve a imagem do contorno atual
        
        if i in range(1, 6): 
            imagem_cont.append(contorno_imagem)

    imagem_combinada = np.zeros_like(imagem_nitidez)
    for imagem in imagem_cont:
        imagem_combinada = cv2.add(imagem_combinada, imagem)

    altura, largura = imagem_combinada.shape[:2]

    contornos_combinada, _ = cv2.findContours(imagem_combinada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    y_max = 0
    x_total = 0
    yi = 600
    xi = 600

    for cont in contornos_combinada: 
    # Crie uma nova imagem com a altura e largura do contorno da imagem combinada
        x, y, largura, altura = cv2.boundingRect(cont)
        xi = min(xi, x)
        yi = min(yi, y)
        x_total += largura
        y_max = max(y_max, y + altura)
    print(xi, yi)
    image_prova = imagem_combinada[yi:y_max, xi:x_total]
    cv2.imshow('imagem.png', imagem_combinada)
    cv2.imshow('imagem', image_prova)

    partes_imagem2 = []

    altura, largura = image_prova.shape
    parte1_x1, parte1_y1, parte1_x2, parte1_y2 = 0, 0, largura // 4, altura
    parte2_x1, parte2_y1, parte2_x2, parte2_y2 = largura // 4, 0, largura // 2, altura
    parte3_x1, parte3_y1, parte3_x2, parte3_y2 = largura // 2, 0, 3 * largura // 4, altura

    # Recorte as três partes
    parte1 = image_prova[parte1_y1:parte1_y2, parte1_x1:parte1_x2]
    parte2 = image_prova[parte2_y1:parte2_y2, parte2_x1:parte2_x2]
    parte3 = image_prova[parte3_y1:parte3_y2, parte3_x1:parte3_x2]

    partes_imagem2.append(parte1)
    partes_imagem2.append(parte2)
    partes_imagem2.append(parte3)

    cv2.imshow(f'img combinada 1', parte1)
    cv2.imshow(f'img combinada 2', parte2)
    cv2.imshow(f'img combinada 3', parte3)

    respostas = []
    partes_imagem3 = []
    altura_parte = altura // 25
    for j in range (0,3):
        for i in range (25):
            if(j == 2 and i == 10):
                break
            calc = (i + 1) * altura_parte + 1
            parte3 = partes_imagem2[j][i * altura_parte: calc, :]
            partes_imagem3.append(parte3)
            cv2.imwrite(f'{j} - {i}.png', parte3)
            largura = parte3.shape[1]
            altura = parte3.shape[0]

            # Defina as faixas de porcentagem
            faixa_A = (0.5, 0.61)
            faixa_B = (0.61, 0.72)
            faixa_C = (0.72, 0.83)
            faixa_D = (0.83, 0.925)
            faixa_E = (0.925, 1)

            # Determine a ROI com base nas faixas de porcentagem
            x1_A, x2_A = int(largura * faixa_A[0]), int(largura * faixa_A[1])
            x1_B, x2_B = int(largura * faixa_B[0]), int(largura * faixa_B[1])
            x1_C, x2_C = int(largura * faixa_C[0]), int(largura * faixa_C[1])
            x1_D, x2_D = int(largura * faixa_D[0]), int(largura * faixa_D[1])
            x1_E, x2_E = int(largura * faixa_E[0]), int(largura * faixa_E[1])

            # Recorte as ROIs com base nas coordenadas calculadas
            roi_A = parte3[:, x1_A:x2_A]
            roi_B = parte3[:, x1_B:x2_B]
            roi_C = parte3[:, x1_C:x2_C]
            roi_D = parte3[:, x1_D:x2_D]
            roi_E = parte3[:, x1_E:x2_E]
            # Exiba ou faça o que for necessário com as ROIs
            cv2.imwrite('ROI A1.png', roi_A)
            cv2.imwrite('ROI B1.png', roi_B)
            cv2.imwrite('ROI C1.png', roi_C)
            cv2.imwrite('ROI D1.png', roi_D)
            cv2.imwrite('ROI E.png', roi_E)

            # Função para detectar círculos pretos em uma ROI
            def detectar_circulos_pretos(roi):
                _, thresh = cv2.threshold(roi, 1, 255, cv2.THRESH_BINARY)
                circulos = cv2.HoughCircles(thresh, cv2.HOUGH_GRADIENT, dp=1,minDist=8,param1=3,   param2=4,   minRadius=3,  maxRadius=10)
                media_cor = np.mean(roi)
                if(media_cor < 160):
                    return circulos
                return None

            # Detecte círculos nas ROIs
            circulos_A = detectar_circulos_pretos(roi_A)
            circulos_B = detectar_circulos_pretos(roi_B)
            circulos_C = detectar_circulos_pretos(roi_C)
            circulos_D = detectar_circulos_pretos(roi_D)
            circulos_E = detectar_circulos_pretos(roi_E)

            # Determine a resposta com base na detecção de círculos
            if circulos_A is not None:
                respostas.append("A")
            elif circulos_B is not None:
                respostas.append("B")
            elif circulos_C is not None:
                respostas.append("C")
            elif circulos_D is not None:
                respostas.append("D")
            elif circulos_E is not None:
                respostas.append("E")
            else:
                respostas.append("Nenhuma resposta detectada")
            print(respostas)
    return respostas

# Diretório da pasta que contém as imagens
# diretorio_pasta = '.\Leitor\provas_realizadas'

# # Lista de arquivos na pasta
# arquivos_na_pasta = os.listdir(diretorio_pasta)
# index = 0

# for arquivo in arquivos_na_pasta:
#     if arquivo.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
#         caminho_arquivo = os.path.join(diretorio_pasta, arquivo)

respostas_A = gabaritoA()

image = cv2.imread('.\Leitor\provas_realizadas\scan0000.jpg')
image_resize = cv2.resize(image, None, fx=0.5, fy=0.5)
image_copy = cv2.resize(image, None, fx=0.5, fy=0.5)

# Converta a imagem para tons de cinza
gray = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray,(1,1),1000)
_, thresh = cv2.threshold(blur, 195, 255, cv2.THRESH_BINARY)

# Detecte os contornos na imagem
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True) 

area_minima_desejada = 10
area_maxima_desejada = 60000

# Filtrar os contornos com área maior que o valor desejado
filtered_countourns = [cont for cont in contours if (cv2.contourArea(cont) > area_minima_desejada) and (cv2.contourArea(cont) < area_maxima_desejada)]

cv2.drawContours(thresh, filtered_countourns, -1, (0, 0, 255), 1)

nova_imagem = np.zeros_like(thresh)

# Desenhe os contornos filtrados na nova imagem
cv2.drawContours(nova_imagem, filtered_countourns, -1, (255, 255, 255), thickness=cv2.FILLED)

# Salve a nova imagem
#cv2.imshow('nova_imagem.png', nova_imagem)

maior_contorno = max(filtered_countourns, key=cv2.contourArea)

# Encontre o ângulo de rotação com base na orientação do maior contorno
(x, y), (largura, altura), angulo = cv2.minAreaRect(maior_contorno)

# Garanta que o ângulo esteja no intervalo de -45 a 45 graus
if angulo < -45:
    angulo += 90

# Rotação da imagem para endireitar
altura, largura = nova_imagem.shape[:2]
rotacao_matriz = cv2.getRotationMatrix2D((largura / 2, altura / 2), angulo, 1)
imagem_endireitada = cv2.warpAffine(nova_imagem, rotacao_matriz, (largura, altura), flags=cv2.INTER_LINEAR)

# Mostre a imagem endireitada
#cv2.imshow('Imagem Endireitada.png', imagem_endireitada)

filtro_nitidez = np.array([[-1, -1, -1],
                        [-1,  9, -1],
                        [-1, -1, -1]])
imagem_nitidez = cv2.filter2D(imagem_endireitada, -1, filtro_nitidez)

_, imagem_binaria = cv2.threshold(imagem_nitidez, 128, 255, cv2.THRESH_BINARY)

contornos, _ = cv2.findContours(imagem_nitidez, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Lista para armazenar os retângulos que atendem aos critérios
retangulos_desejados = []

# Encontre e desenhe as áreas retangulares que atendem aos critérios
for contorno in contornos:
    x, y, largura, altura = cv2.boundingRect(contorno)
    
    if largura >= 30 * altura:
        # Adicione as coordenadas do retângulo desejado à lista
        retangulos_desejados.append((x, y, largura, altura))

# Recorte as áreas dos retângulos desejados da imagem original
areas_recortadas = []
for x, y, largura, altura in retangulos_desejados:
    area_recortada = imagem_nitidez[y:y+altura, x:x+largura]
    areas_recortadas.append(area_recortada)

altura, largura = area_recortada.shape[:2]
comprimento_parte = largura // 5

prova = 0
partes_imagem = []
for i in range(5):
    parte = area_recortada[:, i * comprimento_parte: (i + 1) * comprimento_parte]
    partes_imagem.append(parte)

# Detecte círculos em cada parte da imagem
for i, parte in enumerate(partes_imagem):
    altura1, largura1 = parte.shape
    regiao_interesse = None
    final = False
# Defina a largura da região de interesse como os primeiros 25% da imagem
    match i:
        case 0:
            print('entrou 1')
            largura_regiao_interesse = int(largura1 * 0.20)
            regiao_interesse = parte[:, :largura_regiao_interesse]
        case 1:
            print('entrou 2')
            largura_regiao_interesse = int(largura1 * 0.30)
            regiao_interesse = parte[:, :largura_regiao_interesse]
        case 2:
            print('entrou 3')
            largura_regiao_interesse = int(largura1 * 0.40)
            regiao_interesse = parte[:, :largura_regiao_interesse]
        case 3:
            print('entrou 4')
            largura_regiao_interesse = int(largura1 * 0.45)
            regiao_interesse = parte[:, :largura_regiao_interesse]
        case 4:
            print('entrou 5')
            largura_regiao_interesse = int(largura1 * 0.65)
            regiao_interesse = parte[:, :largura_regiao_interesse]
    # Selecione a região de interesse na imagem
    #cv2.imshow(f'Região de Interesse {i}', regiao_interesse)

    circulos = cv2.HoughCircles(
        regiao_interesse,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=12,  # Distância mínima entre os centros dos círculos
        param1=6,   # Limiar para detecção de bordas
        param2=6,   # Limiar para detecção de círculos
        minRadius=4,  # Raio mínimo do círculo
        maxRadius=8   # Raio máximo do círculo
    )

    if circulos is not None:
        circulos = np.uint16(np.around(circulos))

        # Itere pelos círculos detectados
        for circulo in circulos[0, :]:
            centro_x, centro_y, raio = circulo
            intensidade_cor_centro = None

            if centro_x >= raio and centro_x < regiao_interesse.shape[1] - raio and centro_y >= raio and centro_y < regiao_interesse.shape[0] - raio:
                intensidade_cor_centro = regiao_interesse[centro_y, centro_x]
                cv2.circle(regiao_interesse, (centro_x, centro_y), 2, -1)
                cv2.imshow(f'{i}', regiao_interesse)
            if(intensidade_cor_centro != None):
                print(intensidade_cor_centro)
                #Determine se a intensidade de cor no centro do círculo é próxima ao branco (intensidade alta)
                if intensidade_cor_centro > 250:
                    prova = i + 1
                    final = True
                    break
    if(final):
        break
# print(f'{index}° prova')
print(prova)
# Exemplo de classificação final
if prova == 1:
    print('A imagem é classificada como A')
elif prova == 2:
    print('A imagem é classificada como B')
elif prova == 3:
    print('A imagem é classificada como C')
elif prova == 4:
    print('A imagem é classificada como D')
elif prova == 5:
    print('A imagem é classificada como E')

# Exiba a imagem com as áreas retangulares desenhadas (opcional)
#cv2.imshow('Imagem com Áreas Retangulares', imagem_nitidez)

retangulos_desejados1 = []

imagem_cont = []
nova_img = np.zeros_like(imagem_nitidez)
for i, contorno in enumerate(contornos):
    # Crie uma máscara preta do mesmo tamanho da imagem original
    mask = np.zeros_like(gray)

    # Desenhe o contorno atual na máscara (branco)
    cv2.drawContours(mask, [contorno], 0, 255, thickness=cv2.FILLED)

    # Aplique a máscara à imagem original para obter a imagem do contorno atual
    contorno_imagem = cv2.bitwise_and(imagem_nitidez, imagem_nitidez, mask=mask)
    # Mostre ou salve a imagem do contorno atual
    if i in range(1, 6): 
        imagem_cont.append(contorno_imagem)

imagem_combinada = np.zeros_like(imagem_nitidez)
for imagem in imagem_cont:
    imagem_combinada = cv2.add(imagem_combinada, imagem)

altura, largura = imagem_combinada.shape[:2]
comprimento_parte = largura // 4

contornos_combinada, _ = cv2.findContours(imagem_combinada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
y_max = 0
x_total = 0
yi = 600
xi = 600
index = 0

for cont in contornos_combinada: 
# Crie uma nova imagem com a altura e largura do contorno da imagem combinada
    x, y, largura, altura = cv2.boundingRect(cont)
    xi = min(xi, x)
    yi = min(yi, y)
    x_total += largura
    y_max = max(y_max, y + altura)
print(xi, yi)
image_prova = imagem_combinada[yi:y_max, xi:x_total]
#cv2.imshow('imagem', image_prova)

partes_imagem2 = []

altura, largura = image_prova.shape
parte1_x1, parte1_y1, parte1_x2, parte1_y2 = 0, 0, largura // 4, altura
parte2_x1, parte2_y1, parte2_x2, parte2_y2 = largura // 4, 0, largura // 2, altura
parte3_x1, parte3_y1, parte3_x2, parte3_y2 = largura // 2, 0, 3 * largura // 4, altura

# Recorte as três partes
parte1 = image_prova[parte1_y1:parte1_y2, parte1_x1:parte1_x2]
parte2 = image_prova[parte2_y1:parte2_y2, parte2_x1:parte2_x2]
parte3 = image_prova[parte3_y1:parte3_y2, parte3_x1:parte3_x2]

partes_imagem2.append(parte1)
partes_imagem2.append(parte2)
partes_imagem2.append(parte3)

#cv2.imshow(f'img combinada 1', parte1)
#cv2.imshow(f'img combinada 2', parte2)
#cv2.imshow(f'img combinada 3', parte3)

partes_imagem3 = []
altura_parte = altura // 25
for j in range (0,3):
    for i in range (25):
        if(j == 2 and i == 10):
            break
        calc = (i + 1) * altura_parte + 1
        parte3 = partes_imagem2[j][i * altura_parte: calc, :]
        partes_imagem3.append(parte3)
        #cv2.imwrite(f'{j} - {i}.png', parte3)
        largura = parte3.shape[1]
        altura = parte3.shape[0]

        # Defina as faixas de porcentagem
        faixa_A = (0.5, 0.61)
        faixa_B = (0.61, 0.72)
        faixa_C = (0.72, 0.83)
        faixa_D = (0.83, 0.925)
        faixa_E = (0.925, 1)

        # Determine a ROI com base nas faixas de porcentagem
        x1_A, x2_A = int(largura * faixa_A[0]), int(largura * faixa_A[1])
        x1_B, x2_B = int(largura * faixa_B[0]), int(largura * faixa_B[1])
        x1_C, x2_C = int(largura * faixa_C[0]), int(largura * faixa_C[1])
        x1_D, x2_D = int(largura * faixa_D[0]), int(largura * faixa_D[1])
        x1_E, x2_E = int(largura * faixa_E[0]), int(largura * faixa_E[1])

        # Recorte as ROIs com base nas coordenadas calculadas
        roi_A = parte3[:, x1_A:x2_A]
        roi_B = parte3[:, x1_B:x2_B]
        roi_C = parte3[:, x1_C:x2_C]
        roi_D = parte3[:, x1_D:x2_D]
        roi_E = parte3[:, x1_E:x2_E]
        # Exiba ou faça o que for necessário com as ROIs
        cv2.imwrite('ROI A.png', roi_A)
        cv2.imwrite('ROI B.png', roi_B)
        cv2.imwrite('ROI C.png', roi_C)
        cv2.imwrite('ROI D.png', roi_D)
        cv2.imwrite('ROI E.png', roi_E)

        # Função para detectar círculos pretos em uma ROI
        def detectar_circulos_pretos(roi):
            _, thresh = cv2.threshold(roi, 1, 255, cv2.THRESH_BINARY)
            circulos = cv2.HoughCircles(thresh, cv2.HOUGH_GRADIENT, dp=1,minDist=8,param1=3,   param2=4,   minRadius=3,  maxRadius=10)
            media_cor = np.mean(roi)
            if(media_cor < 160):
                return circulos
            return None

        # Detecte círculos nas ROIs
        circulos_A = detectar_circulos_pretos(roi_A)
        circulos_B = detectar_circulos_pretos(roi_B)
        circulos_C = detectar_circulos_pretos(roi_C)
        circulos_D = detectar_circulos_pretos(roi_D)
        circulos_E = detectar_circulos_pretos(roi_E)

        # Determine a resposta com base na detecção de círculos
        resposta = ""
        if circulos_A is not None:
            resposta = "A"
        elif circulos_B is not None:
            resposta = "B"
        elif circulos_C is not None:
            resposta = "C"
        elif circulos_D is not None:
            resposta = "D"
        elif circulos_E is not None:
            resposta = "E"
        else:
            resposta = "Nenhuma resposta detectada"
        print(resposta)
        #cv2.imwrite(f'img combinada {j} - {i}.png', parte3)

for contorno in contours:
    (x, y, w, h) = cv2.boundingRect(contorno)
    roi = image_resize[y:y+h, x:x+w]
    roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    plt.axis('off')
    plt.imshow(roi_rgb)
    # so ta printando 1 contorno?

    
# index += 1
# cv2.imshow('image_resize', image_resize)
# cv2.imshow('binary_frame', thresh)
# cv2.imshow('image_copy', thresh)


cv2.waitKey(0)
cv2.destroyAllWindows()

82 600


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
